# Fine-Tuning the Financial News Summarization Model

This notebook provides a step-by-step guide to fine-tuning a baseline summarization model for financial news articles. It uses utility functions and classes from the project's `scripts/` directory.

**Sections:**
1. Data Preparation
2. Baseline Model Evaluation
3. Fine-Tuning the Model
4. Evaluation of the Fine-Tuned Model
5. Visual Comparison and Analysis


In [2]:
# Imports
import sys
import os
sys.path.append('../scripts')
from summarization_agent import SummarizationAgent
from finetune_summarizer import SummarizerFineTuner
import matplotlib.pyplot as plt
import json
import numpy as np
from dotenv import load_dotenv
load_dotenv()

True

## 1. Data Preparation

We assume articles are already stored in the vector database. If not, use the relevant data collection scripts to populate the database.

Instantiate the `SummarizationAgent` to load and inspect available articles.

In [ ]:
# Adjust these paths as needed
DB_PATH = '../data/chroma_db'
COLLECTION_NAME = 'financial_articles'
huggingface_api_token = os.environ.get("HUGGINGFACEHUB_API_TOKEN")
agent = SummarizationAgent(db_path=DB_PATH, collection_name=COLLECTION_NAME,huggingface_api_token=huggingface_api_token)

# Example: Retrieve articles for AAPL and TSLA (adjust tickers as needed)
tickers = ['AAPL', 'TSLA']
articles = agent.get_articles_for_portfolio(tickers, days_back=30, article_limit=20)
# print(f'Total articles loaded: {len(articles)}')

2025-05-17 20:08:04,830 - sentence_transformers.SentenceTransformer - INFO - Load pretrained SentenceTransformer: all-MiniLM-L6-v2
2025-05-17 20:08:06,713 - chromadb.telemetry.product.posthog - INFO - Anonymized telemetry enabled. See                     https://docs.trychroma.com/telemetry for more information.
2025-05-17 20:08:06,797 - absl - INFO - Using default tokenizer.


Initializing ChromaDB with persistence directory: ../data/chroma_db
Loaded existing collection 'financial_articles'
Collection 'financial_articles' contains 1034 documents


/Users/giannisalexandrou/Documents/Personal Projects/Trading-Agent/trading-agent/jupyter_files/../scripts/summarization_agent.py:95: LangChainDeprecationWarning: The class `HuggingFaceHub` was deprecated in LangChain 0.0.21 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEndpoint``.
  self.baseline_llm = HuggingFaceHub(
2025-05-17 20:08:06,978 - summarization_agent - INFO - Using HuggingFace BART model as baseline
/Users/giannisalexandrou/Documents/Personal Projects/Trading-Agent/trading-agent/jupyter_files/../scripts/summarization_agent.py:113: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  self.baseline_chain = LLMChain(
2025-05-17 20:08

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Found 20 results. Showing top 3:

Result #1 (Similarity: 0.4080)
Title: Apple Inc. (NASDAQ:AAPL) Position Increased by Schrum Private Wealth Management LLC
Source: Defenseworld Net
Date: 2025-05-16 07:34:55
Chunk: 6 of 6
Link: https://www.defenseworld.net/2025/05/16/apple-inc-nasdaqaapl-position-increased-by-schrum-private-wealth-management-llc.html
Creator: Defense World Staff
Summary: Apple makes up about 5.4% of Schrum Private Wealth Management LLC’s portfolio, making the stock its 3rd biggest position.
Schrum Private Wealth Management LLC’s holdings in Apple were worth $6,365,000 as of its most recent filing with the Securities and Exchange Commission.
Drystone LLC purchased a new stake in shares of Apple in the 4th quarter valued at $25,000.
According to data from MarketBeat, the stock presently has an average rating of “Hold” and an average target price of $235.80.
Insiders sold a total of 191,507 shares of company stock valued at $42,694,080 over the last 90 days.
Article ID: 4d

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Found 20 results. Showing top 3:

Result #1 (Similarity: 0.0635)
Title: Comerica Bank Sells 498 Shares of Heritage Financial Co. (NASDAQ:HFWA)
Source: Defenseworld Net
Date: 2025-05-16 09:02:48
Chunk: 2 of 5
Link: https://www.defenseworld.net/2025/05/16/comerica-bank-sells-498-shares-of-heritage-financial-co-nasdaqhfwa.html
Creator: Defense World Staff
Summary: Finally, Stifel Financial Corp purchased a new stake in shares of Heritage Financial in the fourth quarter worth $205,000.
Heritage Financial Dividend AnnouncementThe business also recently disclosed a quarterly dividend, which will be paid on Wednesday, May 21st.
Heritage Financial Company Profile(Free Report)Heritage Financial Corporation operates as the bank holding company for Heritage Bank that provides various financial services to small and medium sized businesses and individuals in the United States.
Visit HoldingsChannel.com to get the latest 13F filings and insider trades for Heritage Financial Co. (NASDAQ:HFWA – Free 

## 2. Baseline Model Evaluation

Generate summaries using the baseline model and evaluate them with ROUGE metrics.

In [ ]:
articles[0]['document']

In [ ]:
# Generate baseline summaries
articles_with_summaries = agent.generate_summaries(articles)

In [ ]:
agent.huggingface_api_token

In [ ]:
articles_with_summaries[0].keys()

In [ ]:
articles_with_summaries[1]['baseline_summary']

In [ ]:
articles_with_summaries[1]['metadata']

In [ ]:


# Evaluate baseline summaries
baseline_metrics = agent.evaluate_summaries(articles_with_summaries)
print('Baseline ROUGE scores:', baseline_metrics['baseline'])

## 3. Fine-Tuning the Model

We use the `SummarizerFineTuner` class to fine-tune the model on the financial news dataset.

**Note:** Fine-tuning may take a long time and require a GPU. Adjust parameters as needed.

In [ ]:
# Set up fine-tuner
OUTPUT_DIR = '../models/finetuned_summarizer'
fine_tuner = SummarizerFineTuner(
    base_model='facebook/bart-large-cnn',
    db_path=DB_PATH,
    collection_name=COLLECTION_NAME,
    output_dir=OUTPUT_DIR
)

# Fine-tune the model (this may take a while)
finetune_metrics = fine_tuner.finetune(batch_size=4, epochs=3, learning_rate=5e-5)
print('Fine-tuning complete! Metrics:', finetune_metrics)

## 4. Evaluation of the Fine-Tuned Model

Reload the agent to use the fine-tuned model and generate new summaries for evaluation.

In [ ]:
# Reload agent to use fine-tuned model
agent = SummarizationAgent(db_path=DB_PATH, collection_name=COLLECTION_NAME, model_dir=OUTPUT_DIR)

# Generate summaries with the fine-tuned model
finetuned_articles = agent.generate_summaries(articles)

# Evaluate fine-tuned summaries
finetuned_metrics = agent.evaluate_summaries(finetuned_articles)
print('Fine-tuned ROUGE scores:', finetuned_metrics['finetuned'])

## 5. Visual Comparison and Analysis

Let's visualize the ROUGE scores for baseline and fine-tuned models.

In [ ]:
# Plot ROUGE scores
labels = ['ROUGE-1', 'ROUGE-2', 'ROUGE-L']
baseline_scores = [baseline_metrics['baseline']['avg_rouge1'], baseline_metrics['baseline']['avg_rouge2'], baseline_metrics['baseline']['avg_rougeL']]
finetuned_scores = [finetuned_metrics['finetuned']['avg_rouge1'], finetuned_metrics['finetuned']['avg_rouge2'], finetuned_metrics['finetuned']['avg_rougeL']]

x = range(len(labels))
plt.figure(figsize=(8, 5))
plt.bar(x, baseline_scores, width=0.35, label='Baseline', align='center')
plt.bar([i + 0.35 for i in x], finetuned_scores, width=0.35, label='Fine-tuned', align='center')
plt.xticks([i + 0.175 for i in x], labels)
plt.ylabel('ROUGE Score')
plt.title('Baseline vs. Fine-Tuned Model ROUGE Scores')
plt.legend()
plt.show()

---

**Congratulations!** You've now fine-tuned and evaluated your financial news summarization model.

You can further experiment with different models, hyperparameters, and datasets to improve performance.